In [ ]:
# default_exp integrations

In [ ]:
# export
from pyairtable import Table
import pandas as pd
import gspread
from gspread_pandas import Spread
from pathlib import Path
import numpy as np

# Airtable

[Doc link](https://airtable-python-wrapper.readthedocs.io/en/master/)

In [ ]:
# export
def airtable_base_to_df(api_key, base_id, table_name, include_id=False):
    airtable_base = Table(api_key, base_id, table_name)
    records = airtable_base.all()
    
    if include_id == True:
        records_with_id = []
        for r in records:
            record_with_id = {'id': r['id']}
            fields = r['fields']
            record_with_id.update(fields)
            records_with_id.append(record_with_id)
        return pd.DataFrame.from_records(records_with_id)
    else:
        return pd.DataFrame.from_records((r['fields'] for r in records))

In [ ]:
# export
def df_to_airtable_base(df, api_key, base_id, table_name):
    airtable_base = Table(api_key, base_id, table_name)
    airtable_base.batch_create(df.fillna('').to_dict(orient='records'))

In [ ]:
# export
def update_airtable_records(df, api_key, base_id, table_name, record_id_col='id', 
                            replace=False, typecast=False):
    airtable_base = Table(api_key, base_id, table_name)
    records = df.to_dict(orient='records')
    formatted_records = []
    for r in records:
        record_id = r[record_id_col]
        del r[record_id_col]
        formatted_records.append({'id': record_id, 'fields': r})

    airtable_base.batch_update(formatted_records, replace, typecast)

# Google sheet integration

[Doc link](https://gspread-pandas.readthedocs.io/en/latest/index.html)

**Key changes**
1. Get sheet from one command instead of 2 (no reuse of spread object)
2. Do not expose index, and assume no index by default

In [ ]:
# export
def cast_for_gsheets(df):
    # casting as string if not serializable
    for column, dt in zip(df.columns, df.dtypes):
        if dt.type not in [
            np.int64,
            np.float_,
            np.bool_,
        ]:
            df.loc[:, column] = df[column].astype(str)
    return df

In [ ]:
# export
def gsheet_to_df(url, index=None, header_rows=1, start_row=1, unformatted_columns=None, 
                 formula_columns=None, sheet=None, creds=None):
    gsheet = Spread(url, sheet=sheet, creds=creds)
    return gsheet.sheet_to_df(index, header_rows, start_row, unformatted_columns, formula_columns, sheet)

In [ ]:
# export
import gspread
from gspread_pandas.util import fillna
def df_to_gsheet(df, url, append=False, index=True, headers=True, start=(1, 1), replace=False, sheet=None, 
                 raw_column_names=None, raw_columns=None, freeze_index=False, freeze_headers=False, 
                 fill_value='', add_filter=False, merge_headers=False, flatten_headers_sep=None, creds=None):
    if append==False:
        gsheet = Spread(url, sheet=sheet, creds=creds)
        gsheet.df_to_sheet(df, index, headers, start, replace, sheet, 
                     raw_column_names, raw_columns, freeze_index, freeze_headers, 
                     fill_value, add_filter, merge_headers, flatten_headers_sep)
    elif append==True:
        # Fall-back to gspread given there is no high-level function available in gspread pandas
        df = fillna(df, fill_value)
        
        if creds==None:
            scopes = [
                'https://www.googleapis.com/auth/spreadsheets',
                'https://www.googleapis.com/auth/drive'
            ]
            home = str(Path.home())
            gc = gspread.service_account(filename= home + '/.config/gspread_pandas/google_secret.json',  scopes=scopes)
        else:
            gc = gspread.authorize(creds)
        ws = gc.open_by_url(url)
        df = fillna(df, fill_value)
        df = cast_for_gsheets(df).replace('nan','').replace('NaT','').replace('<NA>','').fillna('')
        values = df.values.tolist()
        
        # Handle sheets by name or index
        if type(sheet) == int:
            ss = ws.get_worksheet(sheet)
        elif type(sheet) == str:
            ss = ws.worksheet(sheet)
        
        if ss == None:
            raise Exception('Could not find sheet (tab). Check the sheet number exists (first sheet is 0).')
        else:
            ss.append_rows(values, value_input_option='USER_ENTERED', insert_data_option='INSERT_ROWS')

# Tests

## Airtable

In [ ]:
table = airtable_base_to_df(
    include_id=True,
    base_id = 'appiP2UyUm8OvafTE',
    table_name = 'Applicants',
    api_key = 'keym266zc06VtBemX'
)
                            

In [ ]:
table = table.astype(
    dtype={"YearsOfExperience" : "int64"})

In [ ]:
table["YearsOfExperience"] = table["YearsOfExperience"].subtract(
    other=5)

In [ ]:
table.to_dict(orient='records')

[{'id': 'recLILAVUKdXUvtJ9',
  'Stage': 'Applied',
  'Phone': '(208) 1234-0505',
  'Name': 'Pete Mill',
  'YearsOfExperience': 0,
  'Address': 'Startup avenue 1, ground floor, The Cloud',
  'Email Address': 'pm@gmail.com'},
 {'id': 'recNcGwR3ZuTJnF1l',
  'Stage': 'Applied',
  'Phone': '(646) 555-4389',
  'Name': 'Aristotle',
  'YearsOfExperience': -7,
  'Address': 'Unnamed stret 2, Chalkidiki',
  'Email Address': 'aristotle@makedonia.gr'},
 {'id': 'recsGSkMthmPLL0pb',
  'Stage': 'Applied',
  'Phone': '(865) 123-4567',
  'Name': 'Queen Elizardbeth II',
  'YearsOfExperience': 989,
  'Address': 'Buckingham Palace, London',
  'Email Address': 'hrmqueenlizzy@example.com'}]

In [ ]:
base = Table(base_id = 'appiP2UyUm8OvafTE',
    table_name = 'Applicants',
    api_key = 'keym266zc06VtBemX')

In [ ]:
update_airtable_records(
    table,
    base_id = 'appiP2UyUm8OvafTE',
    table_name = 'Applicants',
    api_key = 'keym266zc06VtBemX')

In [ ]:
table = table.drop(
    columns=["id"])

In [ ]:
df_to_airtable_base(
    table,
    base_id = 'appiP2UyUm8OvafTE',
    table_name = 'Applicants_v2',
    api_key = 'keym266zc06VtBemX')

## Google sheets

In [ ]:
df = gsheet_to_df(url='https://docs.google.com/spreadsheets/d/1tYseM8jnNOBgKf3v9WtLSOCbQNjYw1hABa-cL4nlwds/edit#gid=0',
                  start_row=5,
                  sheet=0)

In [ ]:
df

,user,interactionType,date
0,user006,interaction_1,2020-12-25
1,user006,interaction_2,2020-12-25
2,user007,interaction_1,2020-12-22
3,user005,interaction_1,2020-12-16
4,user010,interaction_1,2020-12-16
5,user005,interaction_2,2020-12-16
6,user006,interaction_1,2020-12-15
7,user002,interaction_1,2020-12-12
8,user007,interaction_1,2020-12-12
9,user001,interaction_2,2020-12-12


In [ ]:
df_to_gsheet(df,
             url='https://docs.google.com/spreadsheets/d/1tYseM8jnNOBgKf3v9WtLSOCbQNjYw1hABa-cL4nlwds/edit#gid=0',
             start=(5,1),
             index=False,
             sheet=1)

In [ ]:
df_to_gsheet(df,
             url='https://docs.google.com/spreadsheets/d/1tYseM8jnNOBgKf3v9WtLSOCbQNjYw1hABa-cL4nlwds/edit#gid=0',
             append=True,
             index=False,
             sheet=1)

### Manual credentials

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_integrations.ipynb.
Converted index.ipynb.
